In [2]:
import requests
import pandas as pd
import json


In [4]:
station_selector = "UKBKB,WBB"
units = "english"
obtimezone = "local"
token = "d25c2abe02b94001a82e7790d9c30f06"

base_url = "https://api.synopticdata.com/v2/stations/latest?"
arguments = [
    f"stid={station_selector}",
    f"units={units}",
    f"obtimezone={obtimezone}",
    f"token={token}",
]
url = base_url + "&".join(arguments)
url


'https://api.synopticdata.com/v2/stations/latest?stid=UKBKB,WBB&units=english&obtimezone=local&token=d25c2abe02b94001a82e7790d9c30f06'

In [6]:
data = json.loads(requests.get(url).text)
data

{'UNITS': {'air_flow_temperature': 'Fahrenheit',
  'wind_direction': 'Degrees',
  'volt': 'volts',
  'precip_accum_five_minute': 'Inches',
  'ozone_concentration': 'ppb',
  'wind_cardinal_direction': 'code',
  'wind_chill': 'Fahrenheit',
  'sensor_error_code': 'code',
  'elevation': 'ft',
  'sea_level_pressure': 'Millibars',
  'heat_index': 'Fahrenheit',
  'precip_accum_one_minute': 'Inches',
  'PM_25_concentration': 'ug/m3',
  'dew_point_temperature': 'Fahrenheit',
  'solar_radiation': 'W/m**2',
  'pressure': 'Millibars',
  'flow_rate': 'liters/min',
  'altimeter': 'INHG',
  'wind_speed': 'knots',
  'air_temp': 'Fahrenheit',
  'wind_gust': 'knots',
  'relative_humidity': '%',
  'qc': 'code',
  'internal_relative_humidity': '%',
  'precip_accum': 'Inches',
  'precip_accum_24_hour': 'Inches',
  'precip_accum_since_local_midnight': 'Inches',
  'position': 'ft'},
 'QC_SUMMARY': {'QC_SHORTNAMES': {'1': 'sl_range_check'},
  'QC_CHECKS_APPLIED': ['sl_range_check'],
  'PERCENT_OF_TOTAL_OBSERV

In [11]:
data["STATION"][0].keys()

dict_keys(['STATUS', 'MNET_ID', 'PERIOD_OF_RECORD', 'ELEVATION', 'NAME', 'STID', 'SENSOR_VARIABLES', 'ELEV_DEM', 'LONGITUDE', 'UNITS', 'STATE', 'OBSERVATIONS', 'RESTRICTED', 'QC_FLAGGED', 'LATITUDE', 'TIMEZONE', 'ID'])

In [42]:
data = json.loads(requests.get(url).text)

Z = {}
for stn in data["STATION"]:
    obs = stn.pop("OBSERVATIONS")
    senvars = stn.pop("SENSOR_VARIABLES")

    metadata_table = pd.DataFrame(stn).T.sort_index()

    value_table = pd.DataFrame(obs).T.sort_index()
    value_table["unit"] = [
        data["UNITS"]["_".join(i.split("_")[:-2])] for i in value_table.index
    ]
    value_table = value_table[["value", "unit", "date_time"]]
    value_table["date_time"] = pd.to_datetime(value_table["date_time"])
    value_table

    Z[stn["STID"]] = (metadata_table, value_table)

In [38]:
Z["WBB"][0]["start"]["NAME"]

'U of U William Browning Building'

In [18]:
data["UNITS"]


{'air_flow_temperature': 'Fahrenheit',
 'wind_direction': 'Degrees',
 'volt': 'volts',
 'precip_accum_five_minute': 'Inches',
 'ozone_concentration': 'ppb',
 'wind_cardinal_direction': 'code',
 'wind_chill': 'Fahrenheit',
 'sensor_error_code': 'code',
 'elevation': 'ft',
 'sea_level_pressure': 'Millibars',
 'heat_index': 'Fahrenheit',
 'precip_accum_one_minute': 'Inches',
 'PM_25_concentration': 'ug/m3',
 'dew_point_temperature': 'Fahrenheit',
 'solar_radiation': 'W/m**2',
 'pressure': 'Millibars',
 'flow_rate': 'liters/min',
 'altimeter': 'INHG',
 'wind_speed': 'knots',
 'air_temp': 'Fahrenheit',
 'wind_gust': 'knots',
 'relative_humidity': '%',
 'qc': 'code',
 'internal_relative_humidity': '%',
 'precip_accum': 'Inches',
 'precip_accum_24_hour': 'Inches',
 'precip_accum_since_local_midnight': 'Inches',
 'position': 'ft'}

In [27]:
data["STATION"][0].keys()


dict_keys(['STATUS', 'MNET_ID', 'PERIOD_OF_RECORD', 'ELEVATION', 'NAME', 'STID', 'SENSOR_VARIABLES', 'ELEV_DEM', 'LONGITUDE', 'UNITS', 'STATE', 'OBSERVATIONS', 'RESTRICTED', 'QC_FLAGGED', 'LATITUDE', 'TIMEZONE', 'ID'])

In [57]:
# Function to convert timedelta to human-readable format
def format_timedelta(td):
    minutes = td.total_seconds() // 60
    if minutes < 60:
        return f"{int(minutes)} minutes ago"
    hours = minutes // 60
    if hours < 24:
        return f"{int(hours)} hours ago"
    days = hours // 24
    return f"{int(days)} days ago"


value_table["time_ago"] = (pd.Timestamp.utcnow() - value_table["date_time"]).apply(
    format_timedelta
)
value_table.loc[~value_table.time_ago.str.contains("days")]


,value,unit,date_time,time_ago
air_temp_value_1,64.0,Fahrenheit,2023-07-08 08:00:00-06:00,36 minutes ago
altimeter_value_1,29.49,INHG,2023-07-08 08:00:00-06:00,36 minutes ago
dew_point_temperature_value_1d,40.93,Fahrenheit,2023-07-08 08:00:00-06:00,36 minutes ago
precip_accum_24_hour_value_1,0.0,Inches,2023-07-08 08:00:00-06:00,36 minutes ago
precip_accum_since_local_midnight_value_1,0.0,Inches,2023-07-08 08:00:00-06:00,36 minutes ago
pressure_value_1d,839.14,Millibars,2023-07-08 08:00:00-06:00,36 minutes ago
relative_humidity_value_1,43.0,%,2023-07-08 08:00:00-06:00,36 minutes ago
sea_level_pressure_value_1d,991.41,Millibars,2023-07-08 08:00:00-06:00,36 minutes ago
wind_cardinal_direction_value_1d,E,code,2023-07-08 08:00:00-06:00,36 minutes ago
wind_direction_value_1,89.0,Degrees,2023-07-08 08:00:00-06:00,36 minutes ago


In [49]:
pd.Timestamp.utcnow()

Timestamp('2023-07-08 14:20:52.089399+0000', tz='UTC')

In [48]:
value_table["date_time"].iloc[0].tz_localize("UTC")


TypeError: Cannot localize tz-aware Timestamp, use tz_convert for conversions

In [60]:
Z["WBB"][0]
states = {df["start"].get("STATE") for _, (df, _) in Z.items()}
states

{'UT'}